In [ ]:
#This deep learning model was made with the intention for use as one of my two models, but GridSearch can't be used with it, and
#attempts were made to use it with the keras-tuner package, but I had some issues with that package (possibly with importing it)
#Not sure if this model can be graded, due to not being able to save it, but it is included anyway.

In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\jetts\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib


In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
selected_features = df.loc[:,['ra', 'dec', 'koi_duration', 'koi_insol', 'koi_period']]
X = selected_features
y = df.loc[:,'koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [5]:
# Scale your data
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
X_scaler = MinMaxScaler(feature_range=(0,5)).fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
#Encoding y data
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

3563         CANDIDATE
4099         CONFIRMED
5460         CANDIDATE
1091         CONFIRMED
5999         CANDIDATE
             ...      
905          CONFIRMED
5192    FALSE POSITIVE
3980    FALSE POSITIVE
235          CONFIRMED
5157         CONFIRMED
Name: koi_disposition, Length: 5243, dtype: object


In [7]:
#One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
#checking one-hot encoding
y_train_categorical

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

# Train the Model



In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
#Input Layer
model.add(Dense(units=250, activation='relu', input_dim=5))
#Hidden Layer
model.add(Dense(units=250, activation='relu'))
#Output layer
model.add(Dense(units=3, activation='softmax'))

In [10]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 250)               1500      
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 753       
Total params: 65,003
Trainable params: 65,003
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs= 90,
    shuffle=True,
    verbose = 2
)

Epoch 1/90
164/164 - 0s - loss: 1.0119 - accuracy: 0.4976
Epoch 2/90
164/164 - 0s - loss: 0.9864 - accuracy: 0.5081
Epoch 3/90
164/164 - 0s - loss: 0.9636 - accuracy: 0.5222
Epoch 4/90
164/164 - 0s - loss: 0.9463 - accuracy: 0.5337
Epoch 5/90
164/164 - 0s - loss: 0.9320 - accuracy: 0.5409
Epoch 6/90
164/164 - 0s - loss: 0.9229 - accuracy: 0.5516
Epoch 7/90
164/164 - 0s - loss: 0.9097 - accuracy: 0.5598
Epoch 8/90
164/164 - 0s - loss: 0.9015 - accuracy: 0.5701
Epoch 9/90
164/164 - 0s - loss: 0.8898 - accuracy: 0.5735
Epoch 10/90
164/164 - 0s - loss: 0.8822 - accuracy: 0.5760
Epoch 11/90
164/164 - 0s - loss: 0.8777 - accuracy: 0.5865
Epoch 12/90
164/164 - 0s - loss: 0.8779 - accuracy: 0.5768
Epoch 13/90
164/164 - 0s - loss: 0.8731 - accuracy: 0.5827
Epoch 14/90
164/164 - 0s - loss: 0.8655 - accuracy: 0.5899
Epoch 15/90
164/164 - 0s - loss: 0.8643 - accuracy: 0.5863
Epoch 16/90
164/164 - 0s - loss: 0.8644 - accuracy: 0.5867
Epoch 17/90
164/164 - 0s - loss: 0.8590 - accuracy: 0.5854
Epoch 

In [13]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}") 

55/55 - 0s - loss: 0.9410 - accuracy: 0.5755
Loss: 0.9409870505332947, Accuracy: 0.5755148530006409


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

# Save the Model

In [16]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'dillon_deep_learning.sav'
joblib.dump(model, filename)
#joblib import failing, checked in terminal and it is installed correctly.

TypeError: cannot pickle 'weakref' object